In [1]:
current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"
import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
ls

 all_dictionaries.pkl                               gold_psy.pkl
 bernardo_35.txt                                    kemper_26.txt
 bundy_34.txt                                       marsh_35.8.txt
 carl_36.txt                                        mitchell_34.txt
 dahmer_23.txt                                      process_transcripts.ipynb
 enriched_dataframe.pkl                             ridgeway_19.txt
'files created on fly that later got formalized'/   transcripts_to_df.pkl
 gacy_27.txt                                        wayne_40.txt


In [3]:
os.chdir(current_dir)

In [5]:
# @title
subjects = []


# Get the current working directory
current_directory = os.getcwd()

# List all .txt files in the current directory and process each file
for file in glob.glob(os.path.join(current_directory, '*.txt')):
    with open(file, 'r') as f:
        file_content = f.read()
    filename_without_extension = os.path.basename(file).split('.txt')[0]
    split_filename = filename_without_extension.split('_')
    name = split_filename[0]
    score = float(split_filename[1])
    data = [name, score, file_content]
    subjects.append(data)

df = pd.DataFrame(subjects, columns=['name', 'score', 'text'])

# Show the DataFrame
print(df.head())

df.to_pickle("transcripts_to_df.pkl")

       name  score                                               text
0     wayne   40.0  \nmy very first contemplation of criminal beha...
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...


In [6]:
needs_enrichment = pd.read_pickle("transcripts_to_df.pkl")

In [7]:
group_dict = {'ridgeway': 'handbook',
 'bundy': 'handbook',
 'kemper': 'handbook',
 'gacy': 'handbook',
 'dahmer': 'handbook',
 'bernardo': 'cbc',
 'mitchell': 'articles',
 'marsh': 'articles',
 'wayne': 'doc',
 'carl': 'doc'}

In [8]:

def clean_text(text):
    # Replace newline characters with a space
    text = text.replace('\n', ' ')

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove non-alphanumeric characters (but keep single whitespaces)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

    # Convert text to lowercase
    return text.lower()


# Example usage

df['clean_text'] = df['text'].apply(clean_text)

In [9]:
def two_tier_score(score):
    if score > 30:
        return "psychopathic"
    else:
        return "not psychopathic"

# Apply the function to the 'Score' column
df['binary'] = df['score'].apply(two_tier_score)

In [10]:
def three_tier_score(score):
    if score >= 30:
        return "high"
    elif 21 <= score < 29:
        return "medium"
    else:
        return "low"

# Apply the function to the 'score' column
df['trinary'] = df['score'].apply(three_tier_score)

In [11]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2024-01-18 15:22:25.782471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 15:22:25.782566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Att

In [12]:
import spacy
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def pos_tag_dataframe(df):
    """
    Function to tag a DataFrame's text column with POS tags and generate new columns for each tag.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing a text column.

    Returns:
    pandas.DataFrame: The original DataFrame with additional columns for each POS tag.
    """
    pos_tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']

    for pos_tag in pos_tags:
        # Create a new column with the tag name and initialize it with empty strings
        df[pos_tag] = ''

    # Iterate over the rows and tag each text with POS tags
    for index, row in df.iterrows():
        doc = nlp(row['clean_text'])  # Assuming 'text' is the name of your text column
        for token in doc:
            if token.pos_ in pos_tags:
                # Append the token text to the corresponding column
                df.at[index, token.pos_] += token.text + ' '

    # Remove trailing spaces
    for pos_tag in pos_tags:
        df[pos_tag] = df[pos_tag].str.strip()

    return df

# Example usage:

rich = pos_tag_dataframe(df)



In [13]:
rich

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,was was s had would was do be is m are can be ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,
1,carl,36.0,I had a statutory rape on a minor fifteen uh t...,i had a statutory rape on a minor fifteen uh t...,psychopathic,high,statutory minor sexual female old good female ...,on for on on around of to at,then someday very at around,was was be is is m be,...,four one,not to na to,i i a she its me it my i i i who i,,,for whether that when that where,,had arrested aggravated be re return gon have m m,,
2,bundy,34.0,\nyes that's true\n\ni grew up in a wonderful ...,yes thats true i grew up in a wonderful home ...,psychopathic,high,true wonderful dedicated young young local loc...,up in with of as of outside in to in of out of...,certainly again more generally more more more ...,do would re might would is would is will is wa...,...,two one five twelve thirteen one twenty thirty...,to nt to to to to nt nt nt nt to not not nt nt...,that i i i what i i i you we our whatever they...,dobson i d god god john q god dobson,,that as that as than once until where that whe...,x,s grew loving mean encountered call think expl...,a,
3,bernardo,35.0,\npaul jason teal aka paul bernardo\n\n\nyeah ...,paul jason teal aka paul bernardo yeah im and...,psychopathic,high,peel regional public peel regional big crazy c...,about with of on for by around on to on with f...,aka forward maybe here just in back there here...,did was did ve been was did did were can was a...,...,seventeen one one one two,nt to not nt nt na to to nt nt nt nt not to no...,i it what that you i i there any i i they they...,paul jason teal paul bernardo september septem...,,that that that that why why if if that that if...,,m makes thought came thought do turned waited ...,,
4,dahmer,23.0,\nhey you're lucky you came up on a\n\n\nday w...,hey youre lucky you came up on a day when the...,not psychopathic,medium,lucky crazy slow steady ordinary last last int...,up on like out of to about to of of off for of...,billy up very just just very so thoroughly jus...,was was was were were was do been been be is i...,...,fourteen fifteen fifteen three eighteen five n...,nt to to not not to not nt nt nt nt to to to n...,you you there it nothing nothing i my it me it...,downtown milwaukee american indian,,when as because how why that why why how where...,,re came s snowing read sent spent reading read...,,
5,gacy,27.0,\n\nwell that's hogwash yeah that's strictly h...,well thats hogwash yeah thats strictly hogwas...,not psychopathic,medium,low last simple homosexual young correct sound...,from in of against in on for to for for for of...,strictly constantly constantly right always aw...,s m m ve were has been ve ve been ve had were ...,...,twelve nineteen seventy eight three million fo...,not to to na nt na nt to not to nt to to nt to...,that that i it i i i i this i its i my it they...,koloz robert piest kozendak december robert ro...,,that that that that that that that that that i...,,hogwash s hogwash told misquoted saying said s...,,
6,kemper,26.0,\nwell i'm not an expert i'm not an authority ...,well im not an expert im not an authority im ...,not psychopathic,medium,more impossible nice ordinary most sick other ...,for than in up up in out of in as of on in out...,almost far that even increasingly back further...,has been would be is did did did m is was wa

In [ ]:
rich.to_pickle("enriched_dataframe.pkl")

In [ ]:
ls

bernardo_35.txt      enriched_dataframe.pkl  mitchell_34.txt            transcripts_to_df.pkl
bow_baselines.ipynb  gacy_27.txt             pos_dataframe.pkl          wayne_40.txt
bundy_34.txt         gold_psy.pkl            process_transcripts.ipynb
carl_36.txt          kemper_26.txt           psychopathy_df.csv
dahmer_23.txt        marsh_34.5.txt          ridgeway_19.txt


In [ ]:
test1 = pd.read_pickle("pos_dataframe.pkl")
test1 = test1.drop('group', axis=1)

In [ ]:
test1 == rich

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
test1.head()

,name,score,text,clean_text,binary,trinary,group,ADJ,ADP,ADV,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,doc,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,
1,carl,36.0,I had a statutory rape on a minor fifteen uh t...,i had a statutory rape on a minor fifteen uh t...,psychopathic,high,doc,statutory minor sexual female old good female ...,on for on on around of to at,then someday very at around,...,four one,not to na to,i i a she its me it my i i i who i,,,for whether that when that where,,had arrested aggravated be re return gon have m m,,
2,bundy,34.0,\nyes that's true\n\ni grew up in a wonderful ...,yes thats true i grew up in a wonderful home ...,psychopathic,high,handbook,true wonderful dedicated young young local loc...,up in with of as of outside in to in of out of...,certainly again more generally more more more ...,...,two one five twelve thirteen one twenty thirty...,to nt to to to to nt nt nt nt to not not nt nt...,that i i i what i i i you we our whatever they...,dobson i d god god john q god dobson,,that as that as than once until where that whe...,x,s grew loving mean encountered call think expl...,a,
3,bernardo,35.0,\npaul jason teal aka paul bernardo\n\n\nyeah ...,paul jason teal aka paul bernardo yeah im and...,psychopathic,high,cbc,peel regional public peel regional big crazy c...,about with of on for by around on to on with f...,aka forward maybe here just in back there here...,...,seventeen one one one two,nt to not nt nt na to to nt nt nt nt not to no...,i it what that you i i there any i i they they...,paul jason teal paul bernardo september septem...,,that that that that why why if if that that if...,,m makes thought came thought do turned waited ...,,
4,dahmer,23.0,\nhey you're lucky you came up on a\n\n\nday w...,hey youre lucky you came up on a day when the...,not psychopathic,medium,handbook,lucky crazy slow steady ordinary last last int...,up on like out of to about to of of off for of...,billy up very just just very so thoroughly jus...,...,fourteen fifteen fifteen three eighteen five n...,nt to to not not to not nt nt nt nt to to to n...,you you there it nothing nothing i my it me it...,downtown milwaukee american indian,,when as because how why that why why how where...,,re came s snowing read sent spent reading read...,,


In [ ]:
are_identical = test1.equals(rich)


In [ ]:
print(are_identical)

True


In [ ]:
test2 = pd.read_csv("psychopathy_df.csv")
test2 = test2.drop('group', axis=1)
test2 = test2.drop('Unnamed: 0', axis=1)
test2 = test2.fillna('')


In [ ]:
test2.head(1)

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,was was s had would was do be is m are can be ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,


In [ ]:
test2

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,was was s had would was do be is m are can be ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,
1,carl,36.0,I had a statutory rape on a minor fifteen uh t...,i had a statutory rape on a minor fifteen uh t...,psychopathic,high,statutory minor sexual female old good female ...,on for on on around of to at,then someday very at around,was was be is is m be,...,four one,not to na to,i i a she its me it my i i i who i,,,for whether that when that where,,had arrested aggravated be re return gon have m m,,
2,bundy,34.0,\nyes that's true\n\ni grew up in a wonderful ...,yes thats true i grew up in a wonderful home ...,psychopathic,high,true wonderful dedicated young young local loc...,up in with of as of outside in to in of out of...,certainly again more generally more more more ...,do would re might would is would is will is wa...,...,two one five twelve thirteen one twenty thirty...,to nt to to to to nt nt nt nt to not not nt nt...,that i i i what i i i you we our whatever they...,dobson i d god god john q god dobson,,that as that as than once until where that whe...,x,s grew loving mean encountered call think expl...,a,
3,bernardo,35.0,\npaul jason teal aka paul bernardo\n\n\nyeah ...,paul jason teal aka paul bernardo yeah im and...,psychopathic,high,peel regional public peel regional big crazy c...,about with of on for by around on to on with f...,aka forward maybe here just in back there here...,did was did ve been was did did were can was a...,...,seventeen one one one two,nt to not nt nt na to to nt nt nt nt not to no...,i it what that you i i there any i i they they...,paul jason teal paul bernardo september septem...,,that that that that why why if if that that if...,,m makes thought came thought do turned waited ...,,
4,dahmer,23.0,\nhey you're lucky you came up on a\n\n\nday w...,hey youre lucky you came up on a day when the...,not psychopathic,medium,lucky crazy slow steady ordinary last last int...,up on like out of to about to of of off for of...,billy up very just just very so thoroughly jus...,was was was were were was do been been be is i...,...,fourteen fifteen fifteen three eighteen five n...,nt to to not not to not nt nt nt nt to to to n...,you you there it nothing nothing i my it me it...,downtown milwaukee american indian,,when as because how why that why why how where...,,re came s snowing read sent spent reading read...,,
5,gacy,27.0,\n\nwell that's hogwash yeah that's strictly h...,well thats hogwash yeah thats strictly hogwas...,not psychopathic,medium,low last simple homosexual young correct sound...,from in of against in on for to for for for of...,strictly constantly constantly right always aw...,s m m ve were has been ve ve been ve had were ...,...,twelve nineteen seventy eight three million fo...,not to to na nt na nt to not to nt to to nt to...,that that i it i i i i this i its i my it they...,koloz robert piest kozendak december robert ro...,,that that that that that that that that that i...,,hogwash s hogwash told misquoted saying said s...,,
6,kemper,26.0,\nwell i'm not an expert i'm not an authority ...,well im not an expert im not an authority im ...,not psychopathic,medium,more impossible nice ordinary most sick other ...,for than in up up in out of in as of on in out...,almost far that even increasingly back further...,has been would be is did did did m is was wa

In [ ]:
are_identical2 = test1.equals(test2)

In [ ]:
test1

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,was was s had would was do be is m are can be ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,
1,carl,36.0,I had a statutory rape on a minor fifteen uh t...,i had a statutory rape on a minor fifteen uh t...,psychopathic,high,statutory minor sexual female old good female ...,on for on on around of to at,then someday very at around,was was be is is m be,...,four one,not to na to,i i a she its me it my i i i who i,,,for whether that when that where,,had arrested aggravated be re return gon have m m,,
2,bundy,34.0,\nyes that's true\n\ni grew up in a wonderful ...,yes thats true i grew up in a wonderful home ...,psychopathic,high,true wonderful dedicated young young local loc...,up in with of as of outside in to in of out of...,certainly again more generally more more more ...,do would re might would is would is will is wa...,...,two one five twelve thirteen one twenty thirty...,to nt to to to to nt nt nt nt to not not nt nt...,that i i i what i i i you we our whatever they...,dobson i d god god john q god dobson,,that as that as than once until where that whe...,x,s grew loving mean encountered call think expl...,a,
3,bernardo,35.0,\npaul jason teal aka paul bernardo\n\n\nyeah ...,paul jason teal aka paul bernardo yeah im and...,psychopathic,high,peel regional public peel regional big crazy c...,about with of on for by around on to on with f...,aka forward maybe here just in back there here...,did was did ve been was did did were can was a...,...,seventeen one one one two,nt to not nt nt na to to nt nt nt nt not to no...,i it what that you i i there any i i they they...,paul jason teal paul bernardo september septem...,,that that that that why why if if that that if...,,m makes thought came thought do turned waited ...,,
4,dahmer,23.0,\nhey you're lucky you came up on a\n\n\nday w...,hey youre lucky you came up on a day when the...,not psychopathic,medium,lucky crazy slow steady ordinary last last int...,up on like out of to about to of of off for of...,billy up very just just very so thoroughly jus...,was was was were were was do been been be is i...,...,fourteen fifteen fifteen three eighteen five n...,nt to to not not to not nt nt nt nt to to to n...,you you there it nothing nothing i my it me it...,downtown milwaukee american indian,,when as because how why that why why how where...,,re came s snowing read sent spent reading read...,,
5,gacy,27.0,\n\nwell that's hogwash yeah that's strictly h...,well thats hogwash yeah thats strictly hogwas...,not psychopathic,medium,low last simple homosexual young correct sound...,from in of against in on for to for for for of...,strictly constantly constantly right always aw...,s m m ve were has been ve ve been ve had were ...,...,twelve nineteen seventy eight three million fo...,not to to na nt na nt to not to nt to to nt to...,that that i it i i i i this i its i my it they...,koloz robert piest kozendak december robert ro...,,that that that that that that that that that i...,,hogwash s hogwash told misquoted saying said s...,,
6,kemper,26.0,\nwell i'm not an expert i'm not an authority ...,well im not an expert im not an authority im ...,not psychopathic,medium,more impossible nice ordinary most sick other ...,for than in up up in out of in as of on in out...,almost far that even increasingly back further...,has been would be is did did did m is was wa

In [ ]:
print(are_identical2)

True
